<a href="https://colab.research.google.com/github/PratikStar/google-colab/blob/main/Analysis_of_embeddings_Full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instructions

Owner: pratik-sutar@g.ecc.u-tokyo.ac.jp

**Inputs (Read-only)**

Timbre Embeddings: 
```
content/drive/MyDrive/utokyo/08. research/Music/VAE/ICASSP/timbre-encoder/model/embeddings-2021-10-05-11-08-20.tsv
content/drive/MyDrive/utokyo/08. research/Music/VAE/ICASSP/timbre-encoder/model/embedding-filenames-2021-10-05-11-08-20.tsv
```

Music Embeddings: 
```
content/drive/MyDrive/utokyo/08. research/Music/VAE/ICASSP/music-encoder/model/embeddings-2021-10-04-06-08-29.tsv
content/drive/MyDrive/utokyo/08. research/Music/VAE/ICASSP/music-encoder/model/embedding-filenames-2021-10-04-06-08-29.tsv
```

Trained VAE
```
/content/drive/MyDrive/Music/VAE/ICASSP/timbre-encoder/
```

**Outputs (write)**

Analysis stuff: 
```
content/drive/MyDrive/research/scripts/analysis/
```
And more...


On IST Node

```!pip install matplotlib google.colab```

# Imports & Setup

In [1]:
import re
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
import inspect
import math
import json
import random

SHOW_LOGS = True
LOG_LEVEL = 5 # {1-6} High Value -> High verbosity

def log(logline, log_level = 1):
    if SHOW_LOGS and log_level <= LOG_LEVEL:
        stack = inspect.stack()
        the_class = None
        if "self" in stack[1][0].f_locals:
            the_class = stack[1][0].f_locals["self"].__class__.__name__
        the_method = stack[1][0].f_code.co_name
        print("{}.{}: {}".format(the_class, the_method, logline))

# Paths

In [31]:
TIMBRE_EMBEDDINGS_PATH = "/content/drive/Shareddrives/timbre-space-drive/03-training/vae/icassp-2021/timbre-vae/model/embeddings-2021-10-05-11-08-20.tsv"
TIMBRE_EMBEDDINGS_FILENAMES_PATH = "/content/drive/Shareddrives/timbre-space-drive/03-training/vae/icassp-2021/timbre-vae/model/embedding-filenames-2021-10-05-11-08-20.tsv"
DICTS_SAVE_PATH = '/content/drive/Shareddrives/timbre-space-drive/data/dicts'

MUSIC_EMBEDDINGS_PATH = "/content/drive/Shareddrives/timbre-space-drive/03-training/vae/icassp-2021/music-vae/model/embeddings-2021-10-04-06-08-29.tsv"
MUSIC_EMBEDDINGS_FILENAMES_PATH = "/content/drive/Shareddrives/timbre-space-drive/03-training/vae/icassp-2021/music-vae/model/embedding-filenames-2021-10-04-06-08-29.tsv"

ANALYSIS_PATH = "/content/drive/Shareddrives/timbre-space-drive/04-analysis"


# Connect to Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Embedding Data loading

### get_segmentid_to_timbre_embedding_dict

In [32]:
# Prerequisite: Drive mounting
#   HARD READS the embeddings from TIMBRE_EMBEDDINGS_PATH, TIMBRE_EMBEDDINGS_FILENAMES_PATH
#   Needs nothing else
def get_segmentid_to_timbre_embedding_dict(save_flg=True, save_path=DICTS_SAVE_PATH):
    d_segmentid_to_timbre_embedding = {}
    count = 0
    with open(TIMBRE_EMBEDDINGS_FILENAMES_PATH, 'r') as keys, open(TIMBRE_EMBEDDINGS_PATH, 'r') as embeddings:
        while True: 
            count += 1
            embedding = embeddings.readline().strip().split('\t')
            if len(embedding) < 4:
                log("Breaking at line: {}, due to embedding. All OK!".format(count))
                break # the length of embedding should be 4
            embedding = list(map(float, embedding))
            key = keys.readline().split(" ")[0]
            d_segmentid_to_timbre_embedding[key] = embedding
            # if line is empty
            # end of file is reached
            if not key:
                log("Breaking at line: {}, due to keys".format(count))
                break
            # print("{}: {}".format(key.strip(), embedding ))
    ## save the dictionary
    if save_flg:
        with open(os.path.join(save_path, 'd_segmentid_to_timbre_embedding.json'), 'wt') as out:
            json.dump(d_segmentid_to_timbre_embedding, out, sort_keys=True, indent=4)
    return d_segmentid_to_timbre_embedding

d_segmentid_to_timbre_embedding = get_segmentid_to_timbre_embedding_dict(save_flg=True)


None.get_segmentid_to_timbre_embedding_dict: Breaking at line: 10801, due to embedding. All OK!


### get_segmentid_to_music_embedding_dict

In [33]:
# Prerequisite: Drive mounting
#   HARD READS the embeddings from MUSIC_EMBEDDINGS_PATH, MUSIC_EMBEDDINGS_FILENAMES_PATH
#   Needs nothing else
# Music encoding containes DI clip embeddings which is not included in Timbre embeddings
def get_segmentid_to_music_embedding_dict(save_flg=True, save_path=DICTS_SAVE_PATH):
    d_segmentid_to_music_embedding = {}
    count = 0
    with open(MUSIC_EMBEDDINGS_FILENAMES_PATH, 'r') as keys, open(MUSIC_EMBEDDINGS_PATH, 'r') as embeddings:
        while True: 
            count += 1
            embedding = embeddings.readline().strip().split('\t')
            if len(embedding) < 4:
                log("Breaking at line: {}, due to embedding. All OK!".format(count))
                break # the length of embedding should be 4
            embedding = list(map(float, embedding))

            key = keys.readline().split(" ")[0]

            d_segmentid_to_music_embedding[key] = embedding
            # if line is empty
            # end of file is reached
            if not key:
                log("Breaking at line: {}, due to keys".format(count))
                break
            # print("{}: {}".format(key.strip(), embedding ))
    ## save the dictionary
    if save_flg:
        with open(os.path.join(save_path, 'd_segmentid_to_music_embedding.json'), 'wt') as out:
            json.dump(d_segmentid_to_music_embedding, out, sort_keys=True, indent=4)
    return d_segmentid_to_music_embedding

d_segmentid_to_music_embedding = get_segmentid_to_music_embedding_dict(save_flg=True)


None.get_segmentid_to_music_embedding_dict: Breaking at line: 10936, due to embedding. All OK!


# Playing with Embeddings


## Passage Embeddings 

### get_passageid_to_list_of_segment_embeddings_dict

In [34]:
# Needs, segment to timbre embedding dict. which can be gotten by get_segment_to_timbre_embedding_dict function.
# This function just needs the dict. no file paths etc required.
def get_passageid_to_list_of_segment_embeddings_dict(d_segmentid_to_timbre_embedding, save_flg=True, save_path=DICTS_SAVE_PATH):
    d_passageid_to_list_of_segment_embeddings = {}
    for key, val in d_segmentid_to_timbre_embedding.items():

        # In our embeddings file, we have the "average" embeddings as per the attributes, we are not interested in that
        if re.search("^avg.*", key):
            continue
        passage = key[:8]
        if passage in d_passageid_to_list_of_segment_embeddings:
            d_passageid_to_list_of_segment_embeddings[passage].append(val)
        else:
            d_passageid_to_list_of_segment_embeddings[passage] = [val]
    ## save the dictionary
    if save_flg:
        with open(os.path.join(save_path, 'd_passageid_to_list_of_segment_embeddings.json'), 'wt') as out:
            json.dump(d_passageid_to_list_of_segment_embeddings, out, sort_keys=True, indent=4)
    return d_passageid_to_list_of_segment_embeddings

d_passageid_to_list_of_segment_embeddings = get_passageid_to_list_of_segment_embeddings_dict(d_segmentid_to_timbre_embedding, save_flg=True)

### [calculate - save]\_variance_of_passage_embeddings\_[list - as_csv]

In [35]:
def calculate_variance_of_passage_embeddings_list(d_passageid_to_list_of_segment_embeddings):
    d_var = {}
    for key, val in d_passageid_to_list_of_segment_embeddings.items():
        d_var[key] = np.var(val, axis=0)
    return d_var

d_passageid_to_variance_of_segment_embeddings = calculate_variance_of_passage_embeddings_list(d_passageid_to_list_of_segment_embeddings)

# HARD saves at ANALYSIS_PATH
def save_variance_of_passage_embeddings_as_csv(d_passageid_to_variance_of_segment_embeddings, save_path=ANALYSIS_PATH):
    file_path = os.path.join(save_path, 'variance_of_passage_embeddings.csv')
    with open(file_path, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["passage", "w-axis", "x-axis", "y-axis", "z-axis"])
        for key, val in d_passageid_to_variance_of_segment_embeddings.items():
            writer.writerow([key] + [str(x) for x in val])

# Uncomment the following line to save as csv
# save_variance_of_passage_embeddings_as_csv(d_passageid_to_variance_of_segment_embeddings)

### [calculate - save]\_mean_of_passage_embeddings\_[list - as_csv]

In [36]:
def calculate_mean_of_passage_embeddings_list(d_passageid_to_list_of_segment_embeddings):
    d_mean = {}
    for key, val in d_passageid_to_list_of_segment_embeddings.items():
        d_mean[key] = np.mean(val, axis=0)

    return d_mean

d_passageid_to_mean_of_segment_embeddings = calculate_mean_of_passage_embeddings_list(d_passageid_to_list_of_segment_embeddings)

# HARD saves at ANALYSIS_PATH
def save_mean_of_passage_embeddings_as_csv(d_passageid_to_mean_of_segment_embeddings, save_path=ANALYSIS_PATH): 
    file_path = os.path.join(save_path, 'mean_of_passage_embeddings.csv')
    with open(file_path, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["passage", "w-axis", "x-axis", "y-axis", "z-axis"])
        for key,val in d_passageid_to_mean_of_segment_embeddings.items():
            writer.writerow([key] + [str(x) for x in val])

# Uncomment the following line to save as csv
# save_mean_of_passage_embeddings_as_csv(d_passageid_to_mean_of_segment_embeddings)

## Clip Embeddings

### get_clipid_to_list_of_segment_embeddings_dict

In [37]:
# Needs, segment to timbre embedding dict. which can be gotten by get_segment_to_timbre_embedding_dict function.
# This function just needs the dict. no file paths etc required.
def get_clipid_to_list_of_segment_embeddings_dict(d_segmentid_to_timbre_embedding, save_flg=True, save_path=DICTS_SAVE_PATH):
    d_clipid_to_list_of_segment_embeddings = {}
    for key, val in d_segmentid_to_timbre_embedding.items():
        clip = key[:5]
        if clip in d_clipid_to_list_of_segment_embeddings:
            d_clipid_to_list_of_segment_embeddings[clip].append(val)
        else:
            d_clipid_to_list_of_segment_embeddings[clip] = [val]
    ## save the dictionary
    if save_flg:
        with open(os.path.join(save_path, 'd_clipid_to_list_of_segment_embeddings.json'), 'wt') as out:
            json.dump(d_passageid_to_list_of_segment_embeddings, out, sort_keys=True, indent=4)
    return d_clipid_to_list_of_segment_embeddings

d_clipid_to_list_of_segment_embeddings = get_clipid_to_list_of_segment_embeddings_dict(d_segmentid_to_timbre_embedding, save_flg=True)

### [calculate - save]\_variance_of_clip_embeddings\_[list - as_csv]

In [38]:
def calculate_variance_of_clip_embeddings_list(d_clipid_to_list_of_segment_embeddings):
    d_var = {}
    for key, val in d_clipid_to_list_of_segment_embeddings.items():
        d_var[key] = np.var(val, axis=0)

    return d_var

d_clipid_to_variance_of_segment_embeddings = calculate_variance_of_clip_embeddings_list(d_clipid_to_list_of_segment_embeddings)

# HARD saves in ANALYSIS_PATH
def save_variance_of_clip_embeddings_as_csv(d_clipid_to_variance_of_segment_embeddings, save_path=ANALYSIS_PATH):
    file_path = os.path.join(save_path, 'variance_of_clip_embeddings.csv')
    with open(file_path, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["passage", "w-axis", "x-axis", "y-axis", "z-axis"])
        for key, val in d_clipid_to_variance_of_segment_embeddings.items():
            writer.writerow([key] + [str(x) for x in val])

# save_variance_of_clip_embeddings_as_csv(d_clipid_to_variance_of_segment_embeddings)

### [calculate - save]\_mean_of_clip_embeddings\_[list - as_csv]

In [39]:
def calculate_mean_of_clip_embeddings_list(d_clipid_to_list_of_segment_embeddings):
    d_mean = {}
    for key, val in d_clipid_to_list_of_segment_embeddings.items():
        d_mean[key] = np.mean(val, axis=0)

    return d_mean

d_clipid_to_mean_of_segment_embeddings = calculate_mean_of_clip_embeddings_list(d_clipid_to_list_of_segment_embeddings)

# HARD saves in ANALYSIS_PATH
def save_mean_of_clip_embeddings_as_csv(d_clipid_to_mean_of_segment_embeddings, save_path=ANALYSIS_PATH):
    file_path = os.path.join(save_path, 'mean_of_clip_embeddings.csv')
    with open(file_path, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["passage", "w-axis", "x-axis", "y-axis", "z-axis"])
        for key,val in d_clipid_to_mean_of_segment_embeddings.items():
            writer.writerow([key] + [str(x) for x in val])

# save_mean_of_clip_embeddings_as_csv(d_clipid_to_mean_of_segment_embeddings)

## visualize_in_2d_any_key_to_embedding_dict

In [40]:
def visualize_in_2d_any_key_to_embedding_dict(d_key_to_embedding, name="Plot", save_path=ANALYSIS_PATH, dpi=120, save_flg=False):
    fig, axs = plt.subplots(nrows=2, ncols=3, constrained_layout=True, figsize=(20, 10), dpi=dpi)
    fig.suptitle(name)
    axs[0][0].scatter([v[0] for v in d_key_to_embedding.values()], [v[1] for v in d_key_to_embedding.values()], s=10, c='red')
    axs[0][0].set_title('w-x plot')

    axs[0][1].scatter([v[0] for v in d_key_to_embedding.values()], [v[2] for v in d_key_to_embedding.values()], s=10, c='blue')
    axs[0][1].set_title('w-y plot')

    axs[0][2].scatter([v[0] for v in d_key_to_embedding.values()], [v[3] for v in d_key_to_embedding.values()], s=10, c='green')
    axs[0][2].set_title('w-z plot')

    axs[1][0].scatter([v[1] for v in d_key_to_embedding.values()], [v[2] for v in d_key_to_embedding.values()], s=10, c='orange')
    axs[1][0].set_title('x-y plot')

    axs[1][1].scatter([v[1] for v in d_key_to_embedding.values()], [v[3] for v in d_key_to_embedding.values()], s=10, c='cyan')
    axs[1][1].set_title('x-z plot')

    axs[1][2].scatter([v[2] for v in d_key_to_embedding.values()], [v[3] for v in d_key_to_embedding.values()], s=10, c='pink')
    axs[1][2].set_title('y-z plot')

    for label in d_key_to_embedding.keys():
        axs[0][0].annotate(label, (d_key_to_embedding[label][0], d_key_to_embedding[label][1]), fontsize=2)
        axs[0][1].annotate(label, (d_key_to_embedding[label][0], d_key_to_embedding[label][2]), fontsize=2)
        axs[0][2].annotate(label, (d_key_to_embedding[label][0], d_key_to_embedding[label][3]), fontsize=2)
        axs[1][0].annotate(label, (d_key_to_embedding[label][1], d_key_to_embedding[label][2]), fontsize=2)
        axs[1][1].annotate(label, (d_key_to_embedding[label][1], d_key_to_embedding[label][3]), fontsize=2)
        axs[1][2].annotate(label, (d_key_to_embedding[label][2], d_key_to_embedding[label][3]), fontsize=2)
    if save_flg:
        plt.savefig(os.path.join(save_path, name +'.png'))
    plt.show()
    plt.close('all') 
    return fig, axs

# fig, axs = visualize_in_2d_any_key_to_embedding_dict(d_clipid_to_mean_of_segment_embeddings, name="mean of clip embeddings", dpi=120, save_flg=False)
# fig, axs = visualize_in_2d_any_key_to_embedding_dict(d_clipid_to_variance_of_segment_embeddings, name="variance of clip embeddings", dpi=120, save_flg=False)
# fig, axs = visualize_in_2d_any_key_to_embedding_dict(d_passageid_to_mean_of_segment_embeddings, name="mean of passage embeddings", dpi=120, save_flg=False)
# fig, axs = visualize_in_2d_any_key_to_embedding_dict(d_passageid_to_variance_of_segment_embeddings, name="variance of passage embeddings", dpi=120, save_flg=False)

# Playing with Annotations

### Imports & setup

In [41]:
import csv
from datetime import datetime
import pytz
import numpy as np
import json
import os
import os.path
import pandas as pd
import pprint
import json
import sys


## Paths

In [42]:
EXPORTS_DYNAMO_DIRECTORY = "/content/drive/Shareddrives/timbre-space-drive/data/exports-dynamo"
TIMBRE_SURVEY_EXPORT = os.path.join(EXPORTS_DYNAMO_DIRECTORY, '20210905160610-timbre_survey.csv')

### get_attribute_to_count_of_pairwise_annotations

In [43]:
# Returns: { <attribute>: count}
# Interpretation: <attribute> was asked in a pairwise question AND the answer chosen (clip_a/clip_b)
def get_attribute_to_count_of_pairwise_annotations(timbre_survey_export, save_flg=True, save_path=DICTS_SAVE_PATH):
    d_attribute_to_count_of_pairwise_annotations = {}
    with open(timbre_survey_export, 'r', newline='')  as f:
        reader = csv.DictReader(f, delimiter=',')
        for row in reader:
            ## The person has not actually listened. TODO: check when the feature was added and filter the data since that date.
            # if len(row['clicks']) == 0:
            #     print('Unclicked annotation')
            #     continue

            ans = row['answer']
            ## The answer is 'Not Applicable'
            if ans not in ['clip_a', 'clip_b']:
                continue
            attribute = row['attribute']

            if attribute not in d_attribute_to_count_of_pairwise_annotations:
                d_attribute_to_count_of_pairwise_annotations[attribute] = 1
            else:
                d_attribute_to_count_of_pairwise_annotations[attribute] += 1


    ## save the dictionary
    if save_flg:
        with open(os.path.join(save_path, 'd_attribute_to_count_of_pairwise_annotations.json'), 'wt') as out:
            json.dump(d_attribute_to_count_of_pairwise_annotations, out, sort_keys=True, indent=4)
    return d_attribute_to_count_of_pairwise_annotations

d_attribute_to_count_of_pairwise_annotations = get_attribute_to_count_of_pairwise_annotations(TIMBRE_SURVEY_EXPORT, save_flg=True)


### get_attribute_to_pairwise_comparision_result_for_arrow_plot

In [45]:
# Returns: { <attribute>: [[clip_a, clip_b], [..., ...]]}
# Here clip_b is the answer
def get_attribute_to_pairwise_comparision_result_for_arrow_plot(timbre_survey_export, save_flg=True, save_path=DICTS_SAVE_PATH):
    d_attribute_to_pairwise_comparision_result = {} 
    with open(timbre_survey_export, 'r', newline='')  as f:
        reader = csv.DictReader(f, delimiter=',')
        for row in reader:
            ## The person has not actually listened. TODO: check when the feature was added and filter the data since that date.
            # if len(row['clicks']) == 0:
            #     print('Unclicked annotation')
            #     continue

            ans = row['answer']
            ## The answer is 'Not Applicable'
            if ans not in ['clip_a', 'clip_b']:
                continue

            attribute = row['attribute']
            ans_clip = row[ans]
            nonans_clip = row['clip_b' if ans == 'clip_a' else 'clip_a']

            if attribute not in d_attribute_to_pairwise_comparision_result:
                d_attribute_to_pairwise_comparision_result[attribute] = []
            d_attribute_to_pairwise_comparision_result[attribute].append(
                [nonans_clip, ans_clip]
            )


    ## save the dictionary
    if save_flg:
        with open(os.path.join(save_path, 'd_attribute_to_pairwise_comparision_result.json'), 'wt') as out:
            json.dump(d_attribute_to_pairwise_comparision_result, out, sort_keys=True, indent=4)
    return d_attribute_to_pairwise_comparision_result

d_attribute_to_pairwise_comparision_result = get_attribute_to_pairwise_comparision_result_for_arrow_plot(TIMBRE_SURVEY_EXPORT, save_flg=True)


### get_passage_vs_attribute_df

In [46]:
def get_passage_vs_attribute_df(timbre_survey_export, save_flg_passage_to_attribute_to_score=True, save_path=DICTS_SAVE_PATH):
    d = {} ## passage -> attribute
    with open(timbre_survey_export, 'r', newline='')  as f:
        reader = csv.DictReader(f, delimiter=',')
        for row in reader:

            ## The person has not actually listened
            # if len(row['clicks']) == 0:
            #     print('Unclicked annotation')
            #     continue

            ans = row['answer']
            ## TODO. Improve the shabby code
            if ans in ['clip_a', 'clip_b']:
                passage = row[ans]

                attribute = row['attribute']

                if row['clip_a'] not in d:
                    d[row['clip_a']] = {}
                    d[row['clip_a']][attribute] = 1 if ans == 'clip_a' else -1
                else:
                    if attribute not in d[row['clip_a']]:
                        d[row['clip_a']][attribute] = 1 if ans == 'clip_a' else -1
                    else:
                        if ans == 'clip_a':
                            d[row['clip_a']][attribute] += 1
                        else:
                            d[row['clip_a']][attribute] -= 1

                if row['clip_b'] not in d:
                    d[row['clip_b']] = {}
                    d[row['clip_b']][attribute] = 1 if ans == 'clip_b' else -1
                else:
                    if attribute not in d[row['clip_b']]:
                        d[row['clip_b']][attribute] = 1 if ans == 'clip_b' else -1
                    else:
                        if ans == 'clip_b':
                            d[row['clip_b']][attribute] += 1
                        else:
                            d[row['clip_b']][attribute] -= 1

            # Processing "others"
            ## The 'others' string is empty
            ## Note: row['others'] is a string
            if len(row['others']) <= 2:
                continue

            attributes = row['others'][1:-1].replace('\'', '').replace(' ', '').split(',')
            
            for attribute in attributes:
                if passage in d:
                    if attribute in d[passage]:
                        d[passage][attribute] += 1
                    else:
                        d[passage][attribute] = 1
                else:
                    d[passage] = {}
                    d[passage][attribute] = 1

    ## Save the dictionary
    if save_flg_passage_to_attribute_to_score:
        with open(os.path.join(save_path, 'd_passage_to_attribute_to_score.json'), 'wt') as out:
            json.dump(d, out, sort_keys=True, indent=4)

    df = pd.DataFrame.from_dict(data=d)
    df = df.T # transpose the DF. Now, rows -> passages, columns -> attributes
    
    # print(df)
    return df

df = get_passage_vs_attribute_df(TIMBRE_SURVEY_EXPORT, save_flg_passage_to_attribute_to_score=True)
# df

### get_attribute_to_passage_to_score_dict

In [47]:
# Input to this will be clips x attributes dataframe. Using get_passage_vs_attribute_df
def get_attribute_to_passage_to_score_dict(df, save_flg=True, save_path=DICTS_SAVE_PATH):
    d_attribute_to_passage_to_score = {}
    for (attribute, passages) in df.iteritems():
        # print(attribute)
        # print(passages)
        d_attribute_to_passage_to_score[attribute] = {}
        for passage, score in passages.items():
            if pd.notnull(score):
                d_attribute_to_passage_to_score[attribute][passage] = score
                # print(passage)
                # print(score)
    ## Save the dictionary
    with open(os.path.join(save_path, 'd_attribute_to_passage_to_score.json'), 'wt') as out:
        json.dump(d_attribute_to_passage_to_score, out, sort_keys=True, indent=4)
    return d_attribute_to_passage_to_score

d_attribute_to_passage_to_score = get_attribute_to_passage_to_score_dict(df, save_flg=True)

### get_attribute_to_min_and_max_score_points_dict

In [48]:
# Calculates the weighted mean of positive score points and negative score points
def get_attribute_to_min_and_max_score_points_dict(d_attribute_to_passage_to_score, d_passage_to_mean_of_segment_embeddings, save_flg=True, save_path=DICTS_SAVE_PATH):
    d_attribute_to_min_and_max_score_points = {}
    for attribute, d_passage_to_score in d_attribute_to_passage_to_score.items():
        d_neg_passages_to_scores = {passageid: score for passageid, score in d_passage_to_score.items() if score < 0}
        d_pos_passages_to_scores = {passageid: score for passageid, score in d_passage_to_score.items() if score > 0} # excluding 0s

        d_neg_passages_to_embeddings = {passageid: em for passageid, em in d_passage_to_mean_of_segment_embeddings.items() if passageid in d_neg_passages_to_scores.keys()}
        d_pos_passages_to_embeddings = {passageid: em for passageid, em in d_passage_to_mean_of_segment_embeddings.items() if passageid in d_pos_passages_to_scores.keys()}

        # calculating neg average
        neg_embeddings = []
        neg_scores = []
        for passageid in d_neg_passages_to_scores:
            neg_scores.append(abs(d_neg_passages_to_scores[passageid]))
            neg_embeddings.append(d_neg_passages_to_embeddings[passageid])
        # log(neg_embeddings)
        # log(neg_scores)
        if len(neg_embeddings) == 0 or len(neg_scores) == 0:
            log("Skipping the passageid: {}".format(passageid))
            break
        average_neg_embedding = np.average(neg_embeddings, weights=neg_scores, axis=0)
        # log(average_neg_embedding)

        # calculating pos average
        pos_embeddings = []
        pos_scores = []
        for passageid in d_pos_passages_to_scores:
            pos_scores.append(abs(d_pos_passages_to_scores[passageid]))
            pos_embeddings.append(d_pos_passages_to_embeddings[passageid])
        # log(pos_embeddings)
        # log(pos_scores)
        if len(pos_embeddings) == 0 or len(pos_scores) == 0:
            log("Skipping the passageid: {}".format(passageid))
            break
        average_pos_embedding = np.average(pos_embeddings, weights=pos_scores, axis=0)
        # log(average_pos_embedding)

        d_attribute_to_min_and_max_score_points[attribute] = {"min": average_neg_embedding.tolist(), "max": average_pos_embedding.tolist()}
        ## Save the dictionary
        with open(os.path.join(save_path, 'd_attribute_to_min_and_max_score_points.json'), 'wt') as out:
            json.dump(d_attribute_to_min_and_max_score_points, out, sort_keys=True, indent=4)
    return d_attribute_to_min_and_max_score_points

d_attribute_to_min_and_max_score_points = get_attribute_to_min_and_max_score_points_dict(d_attribute_to_passage_to_score, d_passageid_to_mean_of_segment_embeddings, save_flg=True)
# d_attribute_to_min_and_max_score_points

### plot_arrow_plot_from_pairwise_comparision_result

In [49]:
ARROW_PLOT_SAVE_PATH = os.path.join(ANALYSIS_PATH, 'visualization/arrow')
def plot_arrow_plot_from_pairwise_comparision_result(attribute, list_of_a_to_b_clips, filename, d_passageid_to_mean_of_segment_embeddings, dpi=120, visualize_flg=False, save_flg=True, save_path=ARROW_PLOT_SAVE_PATH):
    print(attribute)
    # print(list_of_a_to_b_clips)

    a,b = [],[]
    for pair in list_of_a_to_b_clips:
        a.append(d_passageid_to_mean_of_segment_embeddings[pair[0]])
        b.append(d_passageid_to_mean_of_segment_embeddings[pair[1]])

    flat = [x for row in a+b for x in row]
    neglim = -1 * min(flat) if min(flat) < 0 else min(flat)
    poslim = max(flat)
    lim = max(neglim, poslim)

    if not os.path.exists(save_path):
        os.makedirs(save_path)

    fig, axs = plt.subplots(nrows=2, ncols=3, constrained_layout=True, figsize=(20, 10), dpi=dpi)
    fig.suptitle("{}: arrow plot for pairwise annotations results".format(attribute), fontsize='xx-large')
  
    for plot in range(6):
        px = plot // 3
        py = plot % 3
        x = min (px, py)
        y = px + py

        for i in range(len(a)):
            xi = a[i][x]
            xj = a[i][y]
            yi = b[i][x]
            yj = b[i][y]
            # print("For {} {} & {} {}".format(xi, xj, yi, yj))

            axs[px][py].arrow(xi, xj, yi - xi, yj - xj, width = 0.01, head_width=0.1)
            axs[px][py].set_xlim([-1 * lim, lim])
            axs[px][py].set_ylim([-1 * lim, lim])

    if visualize_flg:
        plt.show()
    if save_flg:
        plt.savefig(os.path.join(save_path, filename +'.png'))
    plt.close(fig) 


# Driver: Tested OK! And generated .pngs
# for attribute, pairwise_comparision_result in d_attribute_to_pairwise_comparision_result.items():
#     filename = str(d_attribute_to_count_of_pairwise_annotations[attribute]) + "-" + attribute
#     plot_arrow_plot_from_pairwise_comparision_result(attribute, pairwise_comparision_result, filename, d_passageid_to_mean_of_segment_embeddings, visualize_flg=False, save_flg=True, dpi=300)

# Analysis. The below plots show good results
# Airy Artificial, "Bell-like", "Brilliant", "Brittle", Brutal, buzzy, Clear, Cutting, Dirty, "Distorted"




### plot_negative_vs_positive_passages_by_score_for_given_attribute

In [50]:
POS_NEG_PLOT_SAVE_PATH = os.path.join(ANALYSIS_PATH, 'visualization/pos_neg')
def plot_negative_vs_positive_passages_by_score_for_given_attribute(attribute, d_passageid_to_mean_of_segment_embeddings, d_passage_to_score, dpi=120, visualize_flg=False, save_flg=True, save_path=POS_NEG_PLOT_SAVE_PATH):
    print(attribute)
    print("Number of annotations: {} ".format(len(d_passage_to_score)))
    # print({k: v for k, v in sorted(d_passage_to_score.items(), key=lambda item: item[1], reverse=True)})

    neg_passages = [k for k in d_passage_to_score if d_passage_to_score[k] < 0]
    pos_passages = [k for k in d_passage_to_score if d_passage_to_score[k] >= 0]

    neg_embeddings = [em for k, em in d_passageid_to_mean_of_segment_embeddings.items() if k in neg_passages]
    pos_embeddings = [em for k, em in d_passageid_to_mean_of_segment_embeddings.items() if k in pos_passages]

#   print(neg_embeddings)
#   print(pos_embeddings)

    if not os.path.exists(save_path):
        os.makedirs(save_path)
    fig, axs = plt.subplots(nrows=2, ncols=3, constrained_layout=True, figsize=(20, 10), dpi=dpi)
    fig.suptitle("\"{}\": negative-score (red) v/s positive-score (blue) Annotations Plot. Anno-count: {}".format(attribute, len(d_passage_to_score)), fontsize='xx-large')

    axs[0][0].scatter([v[0] for v in neg_embeddings], [v[1] for v in neg_embeddings], s=10, c='red')
    axs[0][0].scatter([v[0] for v in pos_embeddings], [v[1] for v in pos_embeddings], s=10, c='blue')
    axs[0][0].set_title('w-x plot')

    axs[0][1].scatter([v[0] for v in neg_embeddings], [v[2] for v in neg_embeddings], s=10, c='red')
    axs[0][1].scatter([v[0] for v in pos_embeddings], [v[2] for v in pos_embeddings], s=10, c='blue')
    axs[0][1].set_title('w-y plot')

    axs[0][2].scatter([v[0] for v in neg_embeddings], [v[3] for v in neg_embeddings], s=10, c='red')
    axs[0][2].scatter([v[0] for v in pos_embeddings], [v[3] for v in pos_embeddings], s=10, c='blue')
    axs[0][2].set_title('w-z plot')

    axs[1][0].scatter([v[1] for v in neg_embeddings], [v[2] for v in neg_embeddings], s=10, c='red')
    axs[1][0].scatter([v[1] for v in pos_embeddings], [v[2] for v in pos_embeddings], s=10, c='blue')
    axs[1][0].set_title('x-y plot')

    axs[1][1].scatter([v[1] for v in neg_embeddings], [v[3] for v in neg_embeddings], s=10, c='red')
    axs[1][1].scatter([v[1] for v in pos_embeddings], [v[3] for v in pos_embeddings], s=10, c='blue')
    axs[1][1].set_title('x-z plot')

    axs[1][2].scatter([v[2] for v in neg_embeddings], [v[3] for v in neg_embeddings], s=10, c='red')
    axs[1][2].scatter([v[2] for v in pos_embeddings], [v[3] for v in pos_embeddings], s=10, c='blue')
    axs[1][2].set_title('y-z plot')

    if visualize_flg:
        plt.show()
    if save_flg:
        plt.savefig(os.path.join(save_path, attribute +'.png'))
    plt.close(fig)

# Driver: Tested OK! And generated .pngs
# for attribute, passage_to_score in d_attribute_to_passage_to_score.items():
#     plot_negative_vs_positive_passages_by_score_for_given_attribute(attribute, d_passageid_to_mean_of_segment_embeddings, passage_to_score, dpi=300, visualize_flg=False, save_flg=True)




### Playing with plotting

In [20]:
ANALYSIS_PATH = "/content/drive/Shareddrives/timbre-space-drive/04-analysis"
def visualize_in_2d_list_of_dicts(list_of_d_key_to_embedding, attributes, colors, name="Plot", save_path=ANALYSIS_PATH, annotate=True, dpi=120, save_flg=False):
    if len(list_of_d_key_to_embedding) != len(colors):
        print("len(list_of_d_key_to_embedding) != len(colors)...")
        return

    fig, axs = plt.subplots(nrows=2, ncols=3, constrained_layout=True, figsize=(20, 10), dpi=dpi)
    fig.suptitle(name)
    for i in range(len(colors)):
        axs[0][0].scatter([v[0] for v in list_of_d_key_to_embedding[i].values()], [v[1] for v in list_of_d_key_to_embedding[i].values()], s=10, c=colors[i], label=attributes[i])
        axs[0][0].set_title('w-x plot')
        axs[0][0].legend()

        axs[0][1].scatter([v[0] for v in list_of_d_key_to_embedding[i].values()], [v[2] for v in list_of_d_key_to_embedding[i].values()], s=10, c=colors[i], label=attributes[i])
        axs[0][1].set_title('w-y plot')
        axs[0][1].legend()

        axs[0][2].scatter([v[0] for v in list_of_d_key_to_embedding[i].values()], [v[3] for v in list_of_d_key_to_embedding[i].values()], s=10, c=colors[i], label=attributes[i])
        axs[0][2].set_title('w-z plot')
        axs[0][2].legend()

        axs[1][0].scatter([v[1] for v in list_of_d_key_to_embedding[i].values()], [v[2] for v in list_of_d_key_to_embedding[i].values()], s=10, c=colors[i], label=attributes[i])
        axs[1][0].set_title('x-y plot')
        axs[1][0].legend()

        axs[1][1].scatter([v[1] for v in list_of_d_key_to_embedding[i].values()], [v[3] for v in list_of_d_key_to_embedding[i].values()], s=10, c=colors[i], label=attributes[i])
        axs[1][1].set_title('x-z plot')
        axs[1][1].legend()

        axs[1][2].scatter([v[2] for v in list_of_d_key_to_embedding[i].values()], [v[3] for v in list_of_d_key_to_embedding[i].values()], s=10, c=colors[i], label=attributes[i])
        axs[1][2].set_title('y-z plot')
        axs[1][2].legend()

        if not annotate:
            continue

        for label in list_of_d_key_to_embedding[i].keys():
            axs[0][0].annotate(label, (list_of_d_key_to_embedding[i][label][0], list_of_d_key_to_embedding[i][label][1]), fontsize=2)
            axs[0][1].annotate(label, (list_of_d_key_to_embedding[i][label][0], list_of_d_key_to_embedding[i][label][2]), fontsize=2)
            axs[0][2].annotate(label, (list_of_d_key_to_embedding[i][label][0], list_of_d_key_to_embedding[i][label][3]), fontsize=2)
            axs[1][0].annotate(label, (list_of_d_key_to_embedding[i][label][1], list_of_d_key_to_embedding[i][label][2]), fontsize=2)
            axs[1][1].annotate(label, (list_of_d_key_to_embedding[i][label][1], list_of_d_key_to_embedding[i][label][3]), fontsize=2)
            axs[1][2].annotate(label, (list_of_d_key_to_embedding[i][label][2], list_of_d_key_to_embedding[i][label][3]), fontsize=2)

    # handles, labels = axs[1][2].get_legend_handles_labels()
    # fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, -0.05),
    #       fancybox=True, shadow=True, ncol=4)

    if save_flg:
        plt.savefig(os.path.join(save_path, name +'.png'))
    plt.show()
    # plt.close('all') 
    return fig, axs

attributes = ['All', 'Bright', "Distorted"]
colors = ['yellow', 'red', 'black']
l = []
for attr in attributes:
    if attr.casefold() == "all":
        l.append(d_passageid_to_mean_of_segment_embeddings)
        continue
    attr_passages = [k for k, v in d_attribute_to_passage_to_score[attr].items() if v > 0]
    attr_passages_embeddings = {k: v for k, v in d_passageid_to_mean_of_segment_embeddings.items() if k in attr_passages}
    l.append(attr_passages_embeddings)


# fig, axs = visualize_in_2d_list_of_dicts(l, attributes, colors, name="Attributes: {}".format(", ".join(attributes)), annotate=False, dpi=120, save_flg=True)


# Decoding

## Import Timbre VAE

### downgrade to tf===2.4.0

In [21]:
!pip uninstall --yes tensorflow
!pip install tensorflow==2.4.0

Found existing installation: tensorflow 2.8.0
Uninstalling tensorflow-2.8.0:
  Successfully uninstalled tensorflow-2.8.0
     |████████████████████████████████| 394.7 MB 13 kB/s 
     |████████████████████████████████| 462 kB 43.6 MB/s 
     |████████████████████████████████| 14.8 MB 462 kB/s 
     |████████████████████████████████| 3.8 MB 36.6 MB/s 
     |████████████████████████████████| 132 kB 42.3 MB/s 
     |████████████████████████████████| 2.9 MB 26.3 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68701 sha256=33b53f97c9cca3241d1f6ac2bf70ca038b2087c48056a0d38b48f4b39ed34644
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: numpy
    Found exi

### Import Timbre VAE

In [22]:
import importlib.util
music_vae_spec = importlib.util.spec_from_file_location("music_vae", "/content/drive/Shareddrives/timbre-space-drive/repos/timbre-space/training/vae/music_vae.py")
timbre_vae_spec = importlib.util.spec_from_file_location("timbre_vae", "/content/drive/Shareddrives/timbre-space-drive/repos/timbre-space/training/vae/timbre_vae.py")
music_vae = importlib.util.module_from_spec(music_vae_spec)
timbre_vae = importlib.util.module_from_spec(timbre_vae_spec)
music_vae_spec.loader.exec_module(music_vae)
timbre_vae_spec.loader.exec_module(timbre_vae)


2.4.0


### Load VAE

In [23]:
TIMBREAE_SAVE_PATH = "/content/drive/Shareddrives/timbre-space-drive/03-training/vae/icassp-2021/timbre-vae"
TIMBREAE_MODEL_PATH = os.path.join(TIMBREAE_SAVE_PATH, "model")

WEIGHTS_FILE = "weights.h5"
LEARNING_RATE = 0.0001

timbreae = timbre_vae.VAE.load(TIMBREAE_MODEL_PATH, WEIGHTS_FILE)

timbreae.compile(LEARNING_RATE)
timbreae.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
timbre_input (InputLayer)       [(None, 256, 64, 1)] 0                                            
__________________________________________________________________________________________________
encoder_conv_layer_1 (Conv2D)   (None, 256, 64, 128) 1280        timbre_input[0][0]               
__________________________________________________________________________________________________
encoder_relu_1 (ReLU)           (None, 256, 64, 128) 0           encoder_conv_layer_1[0][0]       
__________________________________________________________________________________________________
encoder_bn_1 (BatchNormalizatio (None, 256, 64, 128) 512         encoder_relu_1[0][0]             
____________________________________________________________________________________________

## segmentid -> spectrogram

### decode_to_spectrogram_from_segment_id

In [24]:
import numpy as np
import os
TIMBREAE_SAVE_PATH = "/content/drive/Shareddrives/timbre-space-drive/03-training/vae/icassp-2021/timbre-vae"
TIMBREAE_MODEL_PATH = os.path.join(TIMBREAE_SAVE_PATH, "model")

def decode_to_spectrogram_from_segment_id(d_segmentid_to_music_embedding, d_segmentid_to_timbre_embedding, segment_id):
    music_embedding = d_segmentid_to_music_embedding[segment_id]
    log("Music Embedding: {}".format(music_embedding), 3)
    timbre_embedding = d_segmentid_to_timbre_embedding[segment_id]
    log("Timbre Embedding: {}".format(timbre_embedding), 3)

    embedding = np.append(timbre_embedding, music_embedding)
    log("Total Embedding: {}".format(embedding), 3)

    spectrogram = timbreae.decoder.predict(embedding[np.newaxis,...])
    spectrogram = spectrogram.reshape(spectrogram.shape[1], spectrogram.shape[2])
    log("Decoded Spectrogram shape: {}".format(spectrogram.shape), 3)

    return spectrogram

# segmentid = "00003-09-09"
# decoded_spectrogram = decode_to_spectrogram_from_segment_id(d_segmentid_to_music_embedding, d_segmentid_to_timbre_embedding, segmentid)


## timbre_embedding -> spectrogram

### decode_to_spectrogram_from_given_timbre_embedding

In [25]:
# This function is useful after moving the timbre embeddings
def decode_to_spectrogram_from_given_timbre_embedding(d_segmentid_to_music_embedding, timbre_embedding, segment_id):
    music_embedding = d_segmentid_to_music_embedding[segment_id]
    log("Music Embedding: {}".format(music_embedding), 3)
    log("Given Timbre Embedding: {}".format(timbre_embedding), 3)

    embedding = np.append(timbre_embedding, music_embedding)
    log("Total Embedding: {}".format(embedding), 3)

    spectrogram = timbreae.decoder.predict(embedding[np.newaxis,...])
    spectrogram = spectrogram.reshape(spectrogram.shape[1], spectrogram.shape[2])
    log("Decoded Spectrogram shape: {}".format(spectrogram.shape), 3)

    return spectrogram

# decoded_spectrogram = decode_to_spectrogram_from_given_timbre_embedding(d_segmentid_to_music_embedding, moved_timbre_embedding, "00003-09-09")

# Audio reconstruction

### segment reconstruction helpers

In [26]:
import soundfile as sf
import librosa
import librosa.display
import matplotlib.pyplot as plt
import re 
import pickle

# for segment
def get_original_spectrogram_from_path(spectrogram_path, segmentid):
    original_spectrogram = None
    for filename in os.listdir(spectrogram_path):
        if re.match("^" + segmentid + ".*\.npy$", filename):
            log("Original spectrogram file : {}".format(filename), 3)
            original_spectrogram = np.load(os.path.join(spectrogram_path, filename))
            log("Shape of og spectrogram: {}".format(original_spectrogram.shape), 3)
    if original_spectrogram is None:
        exit("Original file not found! Exiting")
    return original_spectrogram

# Spectrogram -> Audio
def convert_segment_spectrogram_to_audio_signal(spectrogram, segmentid, save_flg=True, filename="reconstructed-audio", save_path='/content/audio'):
    log("segmentid: {}".format(segmentid), 3)
    log("shape of spectrogram: {}".format(spectrogram.shape), 3)
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    # Denormalize
    with open(os.path.join(MIN_MAX_VALUES_SAVE_DIR, MIN_MAX_VALUES_FILE_NAME), 'rb') as f:
        min_max_values = pickle.load(f)
    min_value, max_value = None, None

    for key, val in min_max_values.items():
        if re.match("^" + segmentid + ".*", key):
            min_value = val["min"]
            max_value = val["max"]
            log("Min: {}, Max: {}".format(str(min_value), str(max_value)), 3)
    if not min_value or not max_value:
        log("Min Max values not found for {}. Returning!".format(segmentid))
        return
    
    denormalized_db_spectrogram = spectrogram * (max_value - min_value) + min_value
    log("shape of denormalized db spectrogram: {}".format(denormalized_db_spectrogram.shape), 3)

    # db_to_amplitude
    inverted_magnitude_spectrogram = librosa.db_to_amplitude(denormalized_db_spectrogram)
    log("shape after db_to_amplitude: {}".format(inverted_magnitude_spectrogram.shape), 3)

    # griffinlim
    reconstructed_segment = librosa.griffinlim(inverted_magnitude_spectrogram, hop_length=HOP_LENGTH)
    log("shape after griffinlim: {}".format(reconstructed_segment.shape), 3)

    if save_flg:
        file_path = os.path.join(save_path, filename + ".wav")
        log("saving to: {}".format(file_path))
        sf.write(file_path, reconstructed_segment, SAMPLE_RATE)
    
    return reconstructed_segment


# Spectrogram -> Image
def convert_spectrogram_to_spectrogramimg(spectrogram, filename, visualize_flg=False, save_flg=True, save_path="/content/"):
    log("Shape of spectrogram img: {}".format(spectrogram.shape))
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    plt.ioff()
    fig, ax = plt.subplots(dpi=120)
    img = librosa.display.specshow(spectrogram, y_axis='log', x_axis='time', ax=ax)

    ax.set_title(filename)
    fig.colorbar(img, ax=ax, format="%+2.2f dB")
    if save_flg:
        fig.savefig(os.path.join(save_path, filename + ".png"))  
    if visualize_flg:
        plt.show()
    plt.close(fig)

def remove_ampid_from_spectrogram(spectrogram):
    return spectrogram[:, 1:]

def get_differential_spectrogram(spectrogram_a, spectrogram_b, filename="differential_spectrogram",visualize_flg=False, save_flg=True, save_path="/content/"):
    if spectrogram_a.shape != spectrogram_b.shape:
        log("Shapes of the spectrogram differ. Cannot calculate the differential")
        return None
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    diff_spectrogram = np.abs(spectrogram_b - spectrogram_a)
    plt.ioff()  
    fig, ax = plt.subplots(dpi=120)
    img = librosa.display.specshow(diff_spectrogram, y_axis='log', x_axis='time', ax=ax)

    ax.set_title(filename)
    fig.colorbar(img, ax=ax, format="%+2.2f dB")
    if save_flg:
        fig.savefig(os.path.join(save_path, filename + ".png"))  
    if visualize_flg:
        plt.show()
    plt.close(fig)
    return diff_spectrogram


### passage reconstruction helpers

In [27]:
# Null Spectrograms: <passage-segment>
# 01-12
# 04-10
# 06-00
# 07-10
# 08-13
# 11-08
# 12-14

def decode_and_construct_passage(passageid, save_flg=True, filename="reconstructed-passage", save_path="/content"):
    # passageid to segmentids
    segmentids = sorted([key for key in d_segmentid_to_music_embedding.keys() if key.startswith(passageid)])
    log("segmentids in passage: {}".format(segmentids), 2)

    passage_signal = np.array([])
    for segmentid in segmentids:
        # get spectrograms of segmentid
        spectrogram = decode_to_spectrogram_from_segment_id(d_segmentid_to_music_embedding, d_segmentid_to_timbre_embedding, segmentid)

        # griffinlim on spectrogram
        reconstructed_signal = convert_segment_spectrogram_to_audio_signal(spectrogram[:,1:], segmentid, save_flg=False)

        # append to passage signal
        passage_signal = np.append(passage_signal, reconstructed_signal)
        # print(spectrogram)

    if save_flg:
        file_path = os.path.join(save_path, filename + ".wav")
        log("saving to: {}".format(file_path))
        sf.write(file_path, passage_signal, SAMPLE_RATE)


# **Segment decoding and reconstruction Pipeline**

In [28]:
sys.exit()
## Driver code: Audio reconstruction for segments
# Config. PLEASE SET THE SEGMENTID
segmentid = "00003-07-04"

# Other Config
SAMPLE_RATE = 22050 # known
FRAME_SIZE = 512 # known
HOP_LENGTH = FRAME_SIZE // 2 # known
SPECTROGRAMS_PATH = "/content/drive/Shareddrives/timbre-space-drive/02-spectrograms-data/spectrogram-2022"
MIN_MAX_VALUES_SAVE_DIR = SPECTROGRAMS_PATH
MIN_MAX_VALUES_FILE_NAME = "min_max_values.pkl"

# 1. segment id -> spectrogram
decoded_spectrogram = decode_to_spectrogram_from_segment_id(d_segmentid_to_music_embedding, d_segmentid_to_timbre_embedding, segmentid)

# 2. spectrogram -> audio

#   just for comparision
original_spectrogram = get_original_spectrogram_from_path(SPECTROGRAMS_PATH, segmentid)

convert_segment_spectrogram_to_audio_signal(original_spectrogram, segmentid, save_flg=True, filename="original-" + segmentid , save_path="/content/")
convert_segment_spectrogram_to_audio_signal(decoded_spectrogram, segmentid, save_flg=True, filename="reconstructed-" + segmentid, save_path="/content/")

# 3. spectrogram -> image
convert_spectrogram_to_spectrogramimg(
    remove_ampid_from_spectrogram(original_spectrogram), 
    "original-" + segmentid, visualize_flg=True, save_flg=True, save_path="/content")
convert_spectrogram_to_spectrogramimg(
    remove_ampid_from_spectrogram(decoded_spectrogram), 
    "decoded-" + segmentid, visualize_flg=True, save_flg=True, save_path="/content")

# 4. differential spectrogram
differential_spectrogram = get_differential_spectrogram(decoded_spectrogram, original_spectrogram, 
                                                        filename="differential-" + segmentid + "-og-recons", 
                                                        visualize_flg=True, save_flg=True, save_path="/content")


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# **Passage decoding and reconstruction Pipeline**

In [ ]:
sys.exit()
passageid = "00003-09"
LOG_LEVEL = 1
# Null Spectrograms: <passage-segment>
# 01-12
# 04-10
# 06-00
# 07-10
# 08-13
# 11-08
# 12-14
decode_and_construct_passage(passageid, save_flg=True, filename="reconstructed-" + passageid, save_path="/content")

# Moving the points

## Helpers

In [ ]:
import numpy as np

def move_point_in_direction_of_vector(point, vector, units_to_move):
    v_hat = vector / np.linalg.norm(vector)
    res = point + v_hat * units_to_move
    return res

## **Moving the segments Pipeline**

In [ ]:
###### PLEASE SET THE FOLLOWING PARAMS ######
SHOW_LOGS=False
save_reconstructed_audio = True

visualize_moved_spectrogram = False
save_moved_spectrogram = True

visualize_og_to_moved_diff_spectrogram = False
save_og_to_moved_diff_spectrogram = True

visualize_prev_to_new_moved_diff_spectrogram = False
save_prev_to_new_moved_diff_spectrogram = True

save_relevant_points_plotted = True
#############################################

# Static Config Here#####
#############################################

# uses a ton of global variables. Name does Not accurately describe the input params.
def pipeline_move_segment(segmentid, attribute, range_of_units_to_move): # get segment embedding
    segment_embedding = d_segmentid_to_timbre_embedding[segmentid]
    log("OG Segment embedding: {}".format(segment_embedding))

    # get original embedding for comparision
    original_spectrogram = get_original_spectrogram_from_path(SPECTROGRAMS_PATH, segmentid)

    # get attribute and min-max
    attribute_min_point = np.array(d_attribute_to_min_and_max_score_points[attribute]['min'])
    attribute_max_point = np.array(d_attribute_to_min_and_max_score_points[attribute]['max'])
    log("Attribute min point: {}".format(attribute_min_point))
    log("Attribute max point: {}".format(attribute_max_point))

    # distance between min-max
    distance_min_max_points = np.linalg.norm(attribute_max_point - attribute_min_point, ord=2)
    log("Distance between min max points: {}".format(distance_min_max_points), log_level=1)
    vector_min_to_max = attribute_max_point - attribute_min_point

    # For plotting the points
    d_moved_points_by_units = {segmentid: segment_embedding, **d_attribute_to_min_and_max_score_points[attribute]}

    prev_spectrogram = original_spectrogram
    prev_units_moved = 0
    number_of_spectrograms = len(range_of_units_to_move)
    for units_to_move in range_of_units_to_move:
        # Move the timbre embeddings
        moved_timbre_embedding = move_point_in_direction_of_vector(segment_embedding, vector_min_to_max, units_to_move)
        log("{} units_moved: {}".format(units_to_move, moved_timbre_embedding))
        d_moved_points_by_units[str(units_to_move)] = moved_timbre_embedding

        # Decode to spectrograms
        moved_decoded_spectrogram = decode_to_spectrogram_from_given_timbre_embedding(d_segmentid_to_music_embedding, moved_timbre_embedding, segmentid)

        # Spectrogram to audio
        moved_audio_segments_save_path = os.path.join("/content/drive/Shareddrives/timbre-space-drive/04-analysis/moving/segments/", segmentid, attribute, "audio")
        filename = "[" + segmentid + " " + attribute + " moved] " + str(units_to_move) + " units"
        convert_segment_spectrogram_to_audio_signal(moved_decoded_spectrogram, segmentid, save_flg=save_reconstructed_audio, filename=filename,
                                                    save_path=moved_audio_segments_save_path)
        # spectrogram -> spectrogramimg
        moved_spectrogramimg_save_path = os.path.join("/content/drive/Shareddrives/timbre-space-drive/04-analysis/moving/segments/", segmentid, attribute)
        convert_spectrogram_to_spectrogramimg(
            remove_ampid_from_spectrogram(moved_decoded_spectrogram), filename=filename, 
            visualize_flg=visualize_moved_spectrogram, save_flg=save_moved_spectrogram,
            save_path=moved_spectrogramimg_save_path)
        
        # OG-moved differential spectrogram imgs
        og_spectrogramimg_diff_save_path = os.path.join(moved_spectrogramimg_save_path, "diff-with-original")
        og_diff_filename = "[" + segmentid + " " + attribute + " diff] OG and " + str(units_to_move) + "units"
        og_to_moved_diff_spectrogram = get_differential_spectrogram(moved_decoded_spectrogram, original_spectrogram, 
                                                            filename=og_diff_filename, 
                                                            visualize_flg=visualize_og_to_moved_diff_spectrogram, 
                                                            save_flg=save_og_to_moved_diff_spectrogram, save_path=og_spectrogramimg_diff_save_path)
        # prev-moved differential spectrogram imgs
        prev_spectrogramimg_diff_save_path = os.path.join(moved_spectrogramimg_save_path, "diff-with-previous")
        diff_filename = "[" + segmentid + " " + attribute + " diff] " + str(prev_units_moved)+" and " + str(units_to_move) + "units"
        prev_to_new_moved_diff_spectrogram = get_differential_spectrogram(moved_decoded_spectrogram, prev_spectrogram, 
                                                            filename=diff_filename, 
                                                            visualize_flg=visualize_prev_to_new_moved_diff_spectrogram, 
                                                            save_flg=save_prev_to_new_moved_diff_spectrogram, save_path=prev_spectrogramimg_diff_save_path)
        prev_spectrogram = moved_decoded_spectrogram
        prev_units_moved = units_to_move

    plt.close('all')

    # Plotting the moved points and the timbre embedding space
    random_keys = random.sample(d_segmentid_to_timbre_embedding.keys(), 1000) # total is about 10.5k
    sampled_points = {k:d_segmentid_to_timbre_embedding[k] for k in random_keys}
    points_to_plot = {**d_moved_points_by_units, **sampled_points}

    print("Plotting {} points".format(len(points_to_plot.keys())))
    visualize_in_2d_any_key_to_embedding_dict(
        points_to_plot, 
        name="Moved segment and sampled timbre embeddings points", dpi=120, save_flg=save_relevant_points_plotted, save_path=moved_spectrogramimg_save_path)

segmentid = "00033-07-05"
attribute = "Clean"
range_of_units_to_move = range(1,10,1)

# Driver
# pipeline_move_segment(segmentid, attribute, range_of_units_to_move)



## **Moving the Passages Pipeline**

In [ ]:
save_reconstructed_audio = True

visualize_original_passage_spectrogram = False
save_original_passage_spectrogram = True

visualize_moved_spectrogram = False
save_moved_spectrogram = True

visualize_og_to_moved_diff_spectrogram = False
save_og_to_moved_diff_spectrogram = True

visualize_prev_to_new_moved_diff_spectrogram = False
save_prev_to_new_moved_diff_spectrogram = True

save_relevant_points_plotted = True

# Other Config
SAMPLE_RATE = 22050 # known
FRAME_SIZE = 512 # known
HOP_LENGTH = FRAME_SIZE // 2 # known
SPECTROGRAMS_PATH = "/content/drive/Shareddrives/timbre-space-drive/02-spectrograms-data/spectrogram-2022"
MIN_MAX_VALUES_SAVE_DIR = SPECTROGRAMS_PATH
MIN_MAX_VALUES_FILE_NAME = "min_max_values.pkl"

SHOW_LOGS=True
LOG_LEVEL = 5
# Null Spectrograms: <passage-segment>
# 01-12
# 04-10
# 06-00
# 07-10
# 08-13
# 11-08
# 12-14

def move_decode_and_construct_passage(attribute, passageid, range_of_units_to_move, save_flg=True):
    # passageid to segmentids
    segmentids = sorted([key for key in d_segmentid_to_music_embedding.keys() if key.startswith(passageid)])
    log("segmentids in passage: {}".format(segmentids), 2)

    # get attribute and min-max
    attribute_min_point = np.array(d_attribute_to_min_and_max_score_points[attribute]['min'])
    attribute_max_point = np.array(d_attribute_to_min_and_max_score_points[attribute]['max'])
    log("Attribute min point: {}".format(attribute_min_point))
    log("Attribute max point: {}".format(attribute_max_point))

    # distance between min-max
    distance_min_max_points = np.linalg.norm(attribute_max_point - attribute_min_point, ord=2)
    log("Distance between min max points: {}".format(distance_min_max_points), log_level=1)
    vector_min_to_max = attribute_max_point - attribute_min_point

    d_for_plotting = d_attribute_to_min_and_max_score_points[attribute]

    prev_passage_spectrogram = None
    prev_units_moved = 0
    for units_to_move in range_of_units_to_move:

        moved_audio_signal = np.array([])
        moved_passage_spectrogram = None
        original_passage_spectrogram = None
        for segmentid in segmentids:
            segment_embedding = d_segmentid_to_timbre_embedding[segmentid]
            d_for_plotting[segmentid + " - OG"] = segment_embedding
            # get original spectrogram for comparision
            original_spectrogram = get_original_spectrogram_from_path(SPECTROGRAMS_PATH, segmentid)[:, 1:]

            # Move the timbre embeddings
            moved_timbre_embedding = move_point_in_direction_of_vector(segment_embedding, vector_min_to_max, units_to_move)
            d_for_plotting["{} - {}units".format(segmentid, units_to_move)] = moved_timbre_embedding
            log("{} moved {} units: {}".format(segmentid, units_to_move, moved_timbre_embedding))

            # Decode to spectrograms
            moved_segment_spectrogram = decode_to_spectrogram_from_given_timbre_embedding(d_segmentid_to_music_embedding, moved_timbre_embedding, segmentid)[:, 1:]

            # Spectrogram to audio
            reconstructed_signal = convert_segment_spectrogram_to_audio_signal(moved_segment_spectrogram, segmentid, save_flg=False)
        
            # append to moved spectrogram
            if moved_passage_spectrogram is None:
                moved_passage_spectrogram = moved_segment_spectrogram
            else:
                moved_passage_spectrogram = np.append(moved_passage_spectrogram, moved_segment_spectrogram, axis=1)

            # append to original spectrogram
            if original_passage_spectrogram is None:
                original_passage_spectrogram = original_spectrogram
            else:
                original_passage_spectrogram = np.append(original_passage_spectrogram, original_spectrogram, axis=1)

            # append to audio segment
            moved_audio_signal = np.append(moved_audio_signal, reconstructed_signal)

        log("{} moved {} units spectrogram shape: {}".format(passageid, units_to_move, moved_passage_spectrogram.shape))
        log("{} reconstructed audio signal shape {}".format(passageid, moved_audio_signal.shape))

        # Save Audio
        moved_audio_segments_save_path = os.path.join("/content/drive/Shareddrives/timbre-space-drive/04-analysis/moving/passages/", passageid, attribute, "audio")
        filename = "[" + passageid + " " + attribute + " moved] " + str(units_to_move) + " units"
        if save_reconstructed_audio:
            if not os.path.exists(moved_audio_segments_save_path):
                os.makedirs(moved_audio_segments_save_path)
            file_path = os.path.join(moved_audio_segments_save_path, filename + ".wav")
            log("saving to: {}".format(file_path))
            sf.write(file_path, moved_audio_signal, SAMPLE_RATE)

        # Save moved spectrogram img
        moved_spectrogramimg_save_path = os.path.join("/content/drive/Shareddrives/timbre-space-drive/04-analysis/moving/passages/", passageid, attribute)
        convert_spectrogram_to_spectrogramimg(
            moved_passage_spectrogram, filename=filename, 
            visualize_flg=visualize_moved_spectrogram, save_flg=save_moved_spectrogram,
            save_path=moved_spectrogramimg_save_path)
        
        # Save original spectrogram img
        original_spectrogramimg_save_path = os.path.join("/content/drive/Shareddrives/timbre-space-drive/04-analysis/moving/passages/", passageid)
        convert_spectrogram_to_spectrogramimg(
            original_passage_spectrogram, filename=passageid, 
            visualize_flg=visualize_original_passage_spectrogram, save_flg=save_original_passage_spectrogram,
            save_path=original_spectrogramimg_save_path)

        ## for calculating differential
        if prev_passage_spectrogram is None:
            prev_passage_spectrogram = original_passage_spectrogram

        # OG-moved differential spectrogram imgs
        og_spectrogramimg_diff_save_path = os.path.join(moved_spectrogramimg_save_path, "diff-with-original")
        og_diff_filename = "[" + passageid + " " + attribute + " diff] OG and " + str(units_to_move) + "units"
        og_to_moved_diff_spectrogram = get_differential_spectrogram(moved_passage_spectrogram, original_passage_spectrogram, 
                                                            filename=og_diff_filename, 
                                                            visualize_flg=visualize_og_to_moved_diff_spectrogram, 
                                                            save_flg=save_og_to_moved_diff_spectrogram, save_path=og_spectrogramimg_diff_save_path)

        # prev-moved differential spectrogram imgs
        prev_spectrogramimg_diff_save_path = os.path.join(moved_spectrogramimg_save_path, "diff-with-previous")
        diff_filename = "[" + passageid + " " + attribute + " diff] " + str(prev_units_moved)+" and " + str(units_to_move) + "units"
        prev_to_new_moved_diff_spectrogram = get_differential_spectrogram(moved_passage_spectrogram, prev_passage_spectrogram, 
                                                            filename=diff_filename, 
                                                            visualize_flg=visualize_prev_to_new_moved_diff_spectrogram, 
                                                            save_flg=save_prev_to_new_moved_diff_spectrogram, save_path=prev_spectrogramimg_diff_save_path)
        prev_passage_spectrogram = moved_passage_spectrogram
        prev_units_moved = units_to_move

    plt.close('all')

    # Plotting the moved points and the timbre embedding space
    # random_keys = random.sample(d_segmentid_to_timbre_embedding.keys(), 1000) # total is about 10.5k
    # sampled_points = {k:d_segmentid_to_timbre_embedding[k] for k in random_keys}
    # points_to_plot = {**d_moved_points_by_units, **sampled_points}

    print("Plotting {} points".format(len(d_for_plotting.keys())))
    visualize_in_2d_any_key_to_embedding_dict(
        d_for_plotting, 
        name="Moved segments embeddings", dpi=120, save_flg=save_relevant_points_plotted, save_path=moved_spectrogramimg_save_path)

passageid = "00033-05"
attribute = "Bright"
range_of_units_to_move = range(1,25,3)

# move_decode_and_construct_passage(attribute, passageid, range_of_units_to_move=range_of_units_to_move)

In [ ]:
!sudo apt-get update 
!sudo apt-get install texlive-xetex texlive-fonts-recommended texlive-generic-recommended
!jupyter nbconvert --to pdf "/content/drive/MyDrive/Colab Notebooks/Analysis of embeddings - Full"